### dataset.csv

#### 檔案大小
 * column : 267欄
 * raw : 116690筆 (11/23 - 12/16)

#### 用法
 * 照日期切割訓練集跟測試集
 * 把 date跟 Y的欄位 drop掉 => X
 * 把想預測的 Y欄位挑出來 => Y

#### Y : 此站點在未來一段時間後的剩餘車輛變化 (8選1) 
 
 * [ sbi_N ] : 該站點N分鐘後的剩餘車數 
 * [ rent_N ] : 該站點接下來N分鐘內的淨借出車數
 * n : 15、30、45、60

#### X : 此站點目前的狀態及其他條件
 
 * [ date ] : 日期
 * [ time ] : (class) 時間
 * [ sna ] : (class) 站點名稱
 * [ weekday ] : (class) 星期幾   
   
 * [ tot ] : 該站點總車位數
 * [ sbi ] : 該站點目前剩餘車數
 * [ sbi_pastN ] : 該站點N分鐘前剩餘車數
 * [ bemp ] : 該站點目前剩餘空位數
 * [ bemp_pastN ] : 該站點N分鐘前剩餘空位數  
   
 * [ place ] : (class) 該站點所屬拖吊地點
 * [ region ] : (class) 該站點所屬拖吊區域  
   
 * [ countPlace ] : 該站點所屬拖吊地點此時段內拖吊車數
 * [ countRegion ] : 該站點所屬拖吊區域此時段內拖吊車數   
   
 * [ pCount_XXX ] : 各拖吊地點此時段內拖吊車數
 * [ rCount_XXX ] : 各拖吊區域此時段內拖吊車數  
   
 * [ temp ] : 目前溫度
 * [ precp ] : 目前降雨量

### UBike 資料集

 * 檔案名稱 : ubikeData_20211124_0000.csv
 * 每15分鐘一份檔案, 每份檔案52筆 (52個站點)
 * 篩選 : 運作中站點 (act = 1)、1122之後

### 拖吊資料集

 * 檔案名稱 : towData.csv
 * 只會有一份檔案，包含期間內所有拖吊資料
 * 篩選 : 1122之後

### 天氣資料集

 * 檔案名稱 : 466920-2021-11-24.csv
 * 一天一份檔案, 24筆資料 (24小時)

## 資料處理

#### 1.  載入套件、讀取檔案

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from numpy import nan as NA
import re
from datetime import datetime, timedelta

In [ ]:
## 合併 UBike檔案
df_ubike_raw = pd.read_csv("../input/Bike_1214.csv")
df_ubike_raw = pd.concat([df_ubike_raw, pd.read_csv("../input/Bike_1217.csv")])
df_ubike_raw.groupby('sna')['date'].count()

df_ubike_raw['time'] = df_ubike_raw['timerange'].apply(lambda x : ("%04d" %x)[:2]+ ":"+ ("%04d" %x)[2:]) # 時間
df_ubike_raw['time'] = df_ubike_raw['time'].apply(lambda x : (datetime.strptime(x, "%H:%M")+ timedelta(minutes=15)).strftime("%H:%M")) # +15 分鐘

df_ubike_raw = df_ubike_raw[df_ubike_raw['act'] == 1] # 篩掉未運作站點
df_ubike_raw = df_ubike_raw.sort_values(by=['sna', 'date', 'time'])
df_ubike_raw.to_csv("../input/ubike_raw.csv", encoding = 'utf_8_sig', index= 0)

In [ ]:
## 拖吊檔案

df_tow_raw = pd.read_csv("../input/TowData.csv").sort_values(by=['date', 'time'])

df_tow_raw['time'] = df_tow_raw['timerange'].apply(lambda x : ("%04d" %x)[:2]+ ":"+ ("%04d" %x)[2:]) # 時間
df_tow_raw['time'] = df_tow_raw['time'].apply(lambda x : (datetime.strptime(x, "%H:%M")+ timedelta(minutes=15)).strftime("%H:%M")) # +15 分鐘

df_tow_raw.to_csv("../input/tow_raw.csv", encoding = 'utf_8_sig', index= 0)

In [ ]:
## 合併天氣檔案

df_weather_raw = DataFrame()
for date in df_ubike_raw['date'].unique().tolist()[:-2] :
    df_weather_now = pd.read_csv("../input/氣溫資料/466920-"+ date.replace("/", "-") + ".csv", skiprows=[1])
    df_weather_now['date'] = df_weather_now['觀測時間(hour)'].apply(lambda x : (datetime.strptime(date, "%Y/%m/%d")+ timedelta(days=1)).strftime("%Y/%m/%d") if x  == 24 else date)
    df_weather_raw = pd.concat([df_weather_raw, df_weather_now])

df_weather_raw['觀測時間(hour)'] = df_weather_now['觀測時間(hour)'].apply(lambda x : 0 if x == 24 else x)
df_weather_raw.to_csv("../input/weather_raw.csv", encoding = 'utf_8_sig', index= 0)

#### 2. Ubike資料

In [ ]:
## 調整、計算欄位

df_ubike_raw = pd.read_csv("../input/ubike_raw.csv").sort_values(by=['sna', 'date', 'time'])
df_ubike = df_ubike_raw.loc[:, ['date', 'time', 'weekday', 'sna', 'place', 'region', 'tot', 'sbi', 'countPlace', 'countRegion']]

df_ubike['sna'] = df_ubike['sna'].apply(lambda x : x.replace("YouBike2.0_", "")) # 簡化站點名稱
df_ubike['place'] = df_ubike['place'].apply(lambda x : x.replace("範圍C區 ", "")) # 簡化拖吊點名稱
df_ubike['place'] = df_ubike['place'].apply(lambda x : x.replace("範圍D區 ", ""))
df_ubike['place'] = df_ubike['place'].apply(lambda x : x.replace("範圍E區 ", ""))
df_ubike['bemp'] = df_ubike['tot'] - df_ubike['sbi'] # 剩餘空位數

 * [ sbi_pastN ] : 該站點N分鐘前剩餘車數
 * [ bemp_pastN ] : 該站點N分鐘前剩餘空位數
 * [ sbi_N ] : 該站點N分鐘後的剩餘車數 
 * [ rent_N ] : 該站點接下來N分鐘內的淨借出車數 

In [ ]:
lag_list = [15, 30, 45, 60]
for lag in range(4) :
    df_ubike['sbi_past'+ str(lag_list[lag])] = df_ubike.groupby('sna')['sbi'].shift(lag+1) # lag 分鐘前的剩餘車數
    df_ubike['bemp_past'+ str(lag_list[lag])] = df_ubike.groupby('sna')['bemp'].shift(lag+1) # lag 分鐘前的剩餘空位
    df_ubike['sbi_'+ str(lag_list[lag])] = df_ubike.groupby('sna')['sbi'].shift(-(lag+1)) # lag 分鐘後的剩餘車數
    df_ubike['rent_'+ str(lag_list[lag])] = df_ubike['sbi'] - df_ubike['sbi_'+ str(lag_list[lag])] # 未來lag 分鐘內的淨借出車數
    

df_ubike = df_ubike[df_ubike['date'].isin(['2021/11/21', '2021/11/22']) == False] # 篩掉 1122前的資料
df_ubike.to_csv("../temp/ubike.csv", encoding = 'utf_8_sig', index= 0)

#### 3. 拖車資料

In [ ]:
## 處理拖吊欄位

df_tow_raw = pd.read_csv("../input/tow_raw.csv").sort_values(by=['date', 'time'])
df_tow_raw = df_tow_raw[df_tow_raw['date'].isin(['2021/11/21', '2021/11/22']) == False] # 篩掉 1122前的資料

df_tow_raw['place'] = df_tow_raw['place'].apply(lambda x : x.replace("範圍E區 ", ""))
df_tow_raw['place'] = df_tow_raw['place'].apply(lambda x : x.replace("範圍D區 ", ""))
df_tow_raw['place'] = df_tow_raw['place'].apply(lambda x : x.replace("範圍C區 ", ""))

In [ ]:
## 計算拖吊次數 (7-10 mins)

def handle_count(x, target, mode) :
    target_date = x['date']
    target_time = x['time']
    count = df_tow_raw[(df_tow_raw[mode] == target) &
           (df_tow_raw['date'] == target_date) &
           (df_tow_raw['time'] == target_time)].count()[0]
    return count

df_tow_count = df_ubike[['date', 'time']]
df_tow_count = df_tow_count[df_tow_count.duplicated() == False]

for place in df_tow_raw['place'].unique().tolist() :
    df_tow_count['pCount_' + place] = df_tow_count.apply(lambda x : handle_count(x, place, 'place'), axis = 1)
for region in df_tow_raw['region'].unique().tolist() :
    df_tow_count['rCount_' + region] = df_tow_count.apply(lambda x : handle_count(x, region, 'region'), axis = 1)

df_tow_count.to_csv("../temp/tow_count.csv", encoding = 'utf_8_sig', index= 0)

#### 4. 天氣資料

In [ ]:
## 做出每15分鐘的資料、挑選欄位

df_weather = pd.read_csv("../input/weather_raw.csv").sort_values(by=['date', '觀測時間(hour)'])
df_weather = df_weather[df_weather['date'].isin(['2021/11/21', '2021/11/22']) == False] # 篩掉 1122前的資料

df_weather['time'] = df_weather['觀測時間(hour)'].apply(lambda x :  "%02d:00" %(x))
df_weather_copy = df_weather*1
for min in [15, 30, 45] :
    df_weather_copy['time'] = df_weather_copy['觀測時間(hour)'].apply(lambda x : "%02d:%02d" %(x, min))
    df_weather = df_weather.append(df_weather_copy)
df_weather['降水量(mm)'] = df_weather['降水量(mm)'].apply(lambda x : 0.1 if x == "T" else x)

df_weather = df_weather.loc[:, ['date', 'time', '氣溫(℃)', '降水量(mm)']].sort_values(by=['date', 'time'])
df_weather.columns = ['date', 'time', 'temp', 'precp']
df_weather.to_csv("../temp/weather.csv", encoding = 'utf_8_sig', index= 0)

#### 5. 合併資料

In [ ]:
## 合併ubike 與拖車資料 
df_ubike = pd.read_csv("../temp/ubike.csv").sort_values(by=['sna', 'date', 'time'])
df_tow_count = pd.read_csv("../temp/tow_count.csv").sort_values(by=['date', 'time'])
df_ubike_tow = pd.merge(df_ubike, df_tow_count)
df_ubike_tow.to_csv("../temp/ubike_tow.csv", encoding = 'utf_8_sig', index= 0)

In [ ]:
## 合併天氣資料
df_ubike_tow = pd.read_csv("../temp/ubike_tow.csv").sort_values(by=['date', 'time'])
df_weather = pd.read_csv("../temp/weather.csv").sort_values(by=['date', 'time'])
df_ubike_tow_weather = pd.merge(df_ubike_tow, df_weather)
df_ubike_tow_weather.to_csv("../temp/ubike_tow_weather.csv", encoding = 'utf_8_sig', index= 0)

#### 6. get_dummies、刪掉出現次數過少的 class欄位(104次)

 * [ time ] : (class) 時間
 * [ sna ] : (class) 站點名稱
 * [ weekday ] : (class) 星期幾
 * [ place ] : (class) 該站點所屬拖吊地點
 * [ region ] : (class) 該站點所屬拖吊區域  

In [ ]:
df_dataset = pd.read_csv("../temp/ubike_tow_weather.csv").sort_values(by=['sna', 'date', 'time'])
class_cols = ["time", "sna", "weekday", "place", "region"]
df_dataset = pd.get_dummies(df_dataset, columns=class_cols , prefix=class_cols)

In [ ]:
## 20秒左右
df_sum = df_dataset.sum()
cols = df_dataset.columns
badCols = []
for col in cols[23:]:
    if df_sum.loc[col] <= 104:
        badCols.append(col)
df_dataset = df_dataset.drop(badCols, axis=1)

#### 7. 排列欄位順序

In [ ]:
x_cols = []
y_cols = []

for target in ['sbi', 'rent'] :
    for lag in [15, 30, 45, 60] :
        y_cols.append(target + "_" + str(lag))

for col in df_dataset.columns.tolist():
    if (col in y_cols) == False :
        x_cols.append(col)

df_dataset = df_dataset.loc[:, x_cols + y_cols].sort_values(by=['date'])
df_dataset.to_csv("../output/dataset.csv", encoding = 'utf_8_sig', index= 0)

In [ ]:
df_dataset[df_dataset['date'] == '2021/11/24'].to_csv("../output/sample_dataset.csv", encoding = 'utf_8_sig', index= 0)

In [ ]:
df_dataset.shape

(106862, 267)